# Indexing Strategies

This notebook demonstrates various indexing strategies for optimizing query performance:
* B-tree indexes
* Partial indexes
* Covering indexes
* Expression indexes
* Index maintenance

## 1. Understanding Existing Indexes

In [ ]:
-- View existing indexes
SELECT 
    schemaname,
    tablename,
    indexname,
    indexdef
FROM pg_indexes
WHERE schemaname = 'public'
ORDER BY tablename, indexname;

In [ ]:
-- Check index usage statistics
SELECT 
    schemaname,
    relname as tablename,
    indexrelname as indexname,
    idx_scan as number_of_scans,
    idx_tup_read as tuples_read,
    idx_tup_fetch as tuples_fetched
FROM pg_stat_user_indexes
WHERE schemaname = 'public'
ORDER BY idx_scan DESC;

## 2. B-tree Index Examples

In [ ]:
-- Single-column B-tree index
CREATE INDEX idx_orders_amount ON orders(total_amount);

-- Query using the index
EXPLAIN ANALYZE
SELECT *
FROM orders
WHERE total_amount > 1000
ORDER BY total_amount DESC
LIMIT 10;

In [ ]:
-- Composite B-tree index
CREATE INDEX idx_orders_customer_date_amount 
ON orders(customer_id, order_date, total_amount);

-- Query using composite index
EXPLAIN ANALYZE
SELECT *
FROM orders
WHERE customer_id = 100
AND order_date >= '2022-01-01'
ORDER BY total_amount DESC;

## 3. Partial Indexes

In [ ]:
-- Create partial index for high-value orders
CREATE INDEX idx_high_value_orders 
ON orders(order_date, customer_id)
WHERE total_amount > 1000;

-- Query using partial index
EXPLAIN ANALYZE
SELECT c.first_name, c.last_name, o.order_date, o.total_amount
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.total_amount > 1000
AND o.order_date >= '2022-01-01';

## 4. Covering Indexes

In [ ]:
-- Create covering index
CREATE INDEX idx_orders_covering ON orders(
    order_date,
    customer_id,
    total_amount,
    status
);

-- Query using covering index
EXPLAIN ANALYZE
SELECT order_date, customer_id, total_amount, status
FROM orders
WHERE order_date BETWEEN '2022-01-01' AND '2022-12-31'
ORDER BY total_amount DESC;

## 5. Expression Indexes

In [ ]:
-- Create expression index
CREATE INDEX idx_customer_lower_email 
ON customers(LOWER(email));

-- Query using expression index
EXPLAIN ANALYZE
SELECT *
FROM customers
WHERE LOWER(email) = 'john.doe@email.com';

## 6. Index Maintenance

In [ ]:
-- Find unused indexes
SELECT 
    schemaname,
    tablename,
    indexname,
    idx_scan,
    pg_size_pretty(pg_relation_size(indexrelid)) as index_size
FROM pg_stat_user_indexes
WHERE idx_scan = 0
AND indexrelname NOT LIKE 'pg_%';

In [ ]:
-- Rebuild index to remove bloat
REINDEX INDEX idx_orders_amount;

## Best Practices for Indexing

1. **Index Selection**
   - Create indexes on frequently queried columns
   - Index foreign key columns
   - Consider column selectivity
   - Use composite indexes for multi-column conditions

2. **Index Types**
   - B-tree for equality and range queries
   - Partial indexes for filtered queries
   - Covering indexes for frequently accessed columns
   - Expression indexes for function-based conditions

3. **Index Maintenance**
   - Monitor index usage
   - Remove unused indexes
   - Rebuild indexes periodically
   - Update statistics regularly

4. **Common Pitfalls**
   - Over-indexing small tables
   - Creating redundant indexes
   - Ignoring index maintenance
   - Not considering write performance